# Check point: KNN from scratch

# Ali LAMINE

Implement KNN from scratch 
The test problem we will be using in this tutorial is the iris classification.

The problem consists of 150 observations of iris flowers from three different species. There are 4 measurements of given flowers: sepal length, sepal width, petal length, and petal width (all in the same unit of centimeters). The predicted attribute is the species, which is either Setosa, Versicolor, or Virginica.

It is a standard dataset where the species is known for all instances. As such we can split the data into training and test datasets and use the results to evaluate our algorithm implementation. Good classification accuracy on this problem is above 90% correct (typically 96% or better).

Save the file in your current working directory with the file name “iris.data“.

This tutorial is broken down into the following steps:

Handle Data: Open the dataset from CSV and split it into test/train datasets.

Similarity: Calculate the distance between two data instances.

Neighbors: Locate k for most similar data instances.

Response: Generate a response from a set of data instances.

Accuracy: Summarize the accuracy of predictions.

Main: Tie it all together.

## 1. Handle Data

In [1]:
#The first thing we need to do is load our data file. 
import csv
with open('iris.data.txt', 'r') as csvfile:
    lines = csv.reader(csvfile)
    for row in lines :
        print (', '.join(row))

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [2]:
#Next we need to split the data into a training dataset 
import csv
import random
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
  with open(filename, 'r') as csvfile:
     lines = csv.reader(csvfile)
     dataset = list(lines)
     for x in range(len(dataset)-1):
         for y in range(4):
             dataset[x][y] = float(dataset[x][y])
         if random.random() < split:
             trainingSet.append(dataset[x])
         else:
             testSet.append(dataset[x])

#We can test this function out with our iris dataset, as follows:
trainingSet=[]
testSet=[]
loadDataset('iris.data.txt', 0.66, trainingSet, testSet)
print ('Train: ' + repr(len(trainingSet)))
print ('Test: ' + repr(len(testSet)) )

Train: 99
Test: 50


## 2. Similarity

To make predictions we need to calculate the similarity between any two given data instances. This is needed so that we can locate the k most similar data instances in the training dataset for a given member of the test dataset and in turn, make a prediction.

Given that all four flower measurements are numeric and have the same units, we can directly use the Euclidean distance measure. 

Additionally, we want to control which fields to include in the distance calculation. Specifically, we only want to include the first 4 attributes. One approach is to limit the Euclidean distance to a fixed length, ignoring the final dimension.

Putting all of this together, you have to define the Euclidean distance

In [3]:
import math

def euclideanDistance(instance1, instance2, length):
    s=0
    for i in range(length):
        s+=math.pow(instance1[i]-instance2[i],2)
    return math.sqrt(s)

data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']

distance = euclideanDistance(data1, data2, 3)

print ('Distance: ' + repr(distance))

Distance: 3.4641016151377544


## 3. Neighbors

Now that we have a similarity measure, we can use it to collect the k most similar instances for a given unseen instance.

This is a straightforward process of calculating the distance for all instances and selecting a subset with the smallest distance values.

Below is the getNeighbors function that returns k most similar neighbors from the training set for a given test instance (using the already defined euclideanDistance function)

In [4]:
import operator

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors


#We can test out this function as follows:

trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getNeighbors(trainSet, testInstance, 1)
print(neighbors)


[[4, 4, 4, 'b']]


## 4. Response

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.

We can do this by allowing each neighbor to vote for their class attribute, and taking the majority vote as the prediction.

Below is a function for getting the majority voted response from a number of neighbors. It assumes the class is the last attribute for each neighbor.

In [5]:
import operator
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

# We can test out this function with some test neighbors, as follows:

neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


This approach returns one response in the case of a draw, but you could handle such cases in a specific way, such as returning no response or selecting an unbiased random response.

## 5. Accuracy

We have all of the pieces of the KNN algorithm in place. An important remaining concern is how to evaluate the accuracy of predictions.

An easy way to evaluate the accuracy of the model is to calculate a ratio of the total correct predictions out of all predictions made, called classification accuracy.

Below is the getAccuracy function that sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [6]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

#We can test this function with a test dataset and predictions, as follows:

testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


## 6. Main

We now have all the elements of the algorithm and  you can put them all in one main function

In [7]:
def knn(k=5):    
    trainingSet=[]
    testSet=[]
    loadDataset('iris.data.txt', 0.66, trainingSet, testSet)
    print ('Train: ' + repr(len(trainingSet)))
    print ('Test: ' + repr(len(testSet)) )
    predictions=[]
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('Predicted is  ' + repr(result) + ' Actual is  ' + repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')

In [8]:
knn()

Train: 100
Test: 49
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted 

In [9]:
for i in range(1,10):
    print("Results for KNN with neighbors =",i)
    knn(k=i)
    print("-----------------------------------------------------")
    print("-----------------------------------------------------")

Results for KNN with neighbors = 1
Train: 114
Test: 35
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is  'Iris-versicolor'
Predicted is  'Iris-versicolor' Actual is

Predicted is  'Iris-virginica' Actual is  'Iris-virginica'
Predicted is  'Iris-virginica' Actual is  'Iris-virginica'
Accuracy: 98.21428571428571%
-----------------------------------------------------
-----------------------------------------------------
Results for KNN with neighbors = 7
Train: 104
Test: 45
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
P

## 7. Another distance metric

In this part, you are asked to define another distance metric in addition to the Euclidean distance.

In [10]:
def ManhattanDistance(instance1, instance2, length):
    s=0
    for i in range(length):
        s+=abs(instance1[i]-instance2[i])
    return s

In [11]:
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']

distance = ManhattanDistance(data1, data2, 3)

print ('Distance: ' + repr(distance))

Distance: 6


In [12]:
import operator
def getNeighborsDist(trainingSet, testInstance, k, dist="euclidean"):
    distances = []
    length = len(testInstance)-1
    if dist=="euclidean":
        for x in range(len(trainingSet)):
            dist = euclideanDistance(testInstance, trainingSet[x], length)
            distances.append((trainingSet[x], dist))
    if dist=="manhattan":
        for x in range(len(trainingSet)):
            dist = ManhattanDistance(testInstance, trainingSet[x], length)
            distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getNeighborsDist(trainSet, testInstance, 1,"manhattan")
print(neighbors)


[[4, 4, 4, 'b']]


In [13]:
def knn(k=5, distance="euclidean"):    
    trainingSet=[]
    testSet=[]
    loadDataset('iris.data.txt', 0.66, trainingSet, testSet)
    print ('Train: ' + repr(len(trainingSet)))
    print ('Test: ' + repr(len(testSet)) )
    predictions=[]
    for x in range(len(testSet)):
        neighbors = getNeighborsDist(trainingSet, testSet[x], k, dist=distance)
        result = getResponse(neighbors)
        predictions.append(result)
        print('Predicted is  ' + repr(result) + ' Actual is  ' + repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print("Number of Neighbors :", k)
    print("Distance Metric : ", distance)
    print('Accuracy: ' + repr(accuracy) + '%')

In [14]:
knn()

Train: 94
Test: 55
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-versico

In [15]:
knn(k=7, distance="manhattan")

Train: 98
Test: 51
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa' Actual is  'Iris-setosa'
Predicted is  'Iris-setosa'